In [ ]:
import pandas as pd
import numpy as np 
import math
import sklearn.metrics
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


In [ ]:
all=pd.read_csv('ai_article_ALL.csv', sep=';')
all=all[['ART_CINV','ART_DESC']]
all.shape

(61176, 2)

In [ ]:
f20=pd.read_csv('20201102.csv', sep=';')
f20.head(2)

,ART_CINV,ART_CINVO,H_DATE,FCT_ERROR,FCT_ERROR_NC,FCT_ERROR_QTY,FCT_VALUE,FCT_VALUE_WEEK_REF,FCT_DATE,DC_CODE,...,nm1,nm2,nm3,valRetenuesSpeD,valRetenuesPromo,valRetenuesHolidays,valRetenuesSansPromo,valRetenuesWeather,ART_SITE_KEY,valForError
0,1907718,1907718,20200727,0.1,0.0,0.0,0.0,0.0,20201102,90009,...,0.0,0.0,0.0,[ ],[ 0 . 0 ],[ ],[ 0 ],[ ],90009_1907718,"|||liens retenus:[1907718]||||[ 0 . 0 , ' - ..."
1,1907718,1907718,20200803,0.1,0.0,0.0,0.0,0.0,20201102,90009,...,0.0,0.0,0.0,[ ],[ ],[ ],[ 0 ],[ ],90009_1907718,"|||liens retenus:[1907718]||||[ 0 . 0 , ' - ..."


In [ ]:
f21=pd.read_csv('20210127.csv', sep=';')
f21.head(2)

,ART_CINV,ART_CINVO,H_DATE,FCT_ERROR,FCT_ERROR_NC,FCT_ERROR_QTY,FCT_VALUE,FCT_VALUE_WEEK_REF,FCT_DATE,DC_CODE,...,nm1,nm2,nm3,valRetenuesSpeD,valRetenuesPromo,valRetenuesHolidays,valRetenuesSansPromo,valRetenuesWeather,ART_SITE_KEY,valForError
0,1907732,1907732,20190729,0.1,0.0,0.0,0.0,0.0,20201102,90009,...,0.0,0.0,0.0,[ ],[ ],[ ],[ 0 ],[ ],90009_1907732,||||||| Second AI model:sansImpact|||| ||||lie...
1,1907732,1907732,20190805,0.1,0.0,0.0,0.0,0.0,20201102,90009,...,0.0,0.0,0.0,[ ],[ ],[ ],[ 0 ],[ ],90009_1907732,||||||| Second AI model:sansCorr|||| ||||liens...


In [ ]:
print(f20.shape, f21.shape)

(175392, 106) (331405, 109)


In [ ]:
f20['ART_CINV'].value_counts()


1907718    27
2397554    27
2498578    27
2490350    27
2486434    27
           ..
2220122    27
2219392    27
2218780    27
2218251    27
2856463    27
Name: ART_CINV, Length: 6496, dtype: int64

In [ ]:
f21['ART_CINV'].value_counts()

1907732    79
2562751    79
2569633    79
2572352    79
2575297    79
           ..
2244223    79
2245309    79
2246383    79
2247399    79
2980508    79
Name: ART_CINV, Length: 4195, dtype: int64

In [ ]:
print(f20['H_DATE'].min(), f20['H_DATE'].max())

20200727 20210125


In [ ]:
print(f21['H_DATE'].min(), f21['H_DATE'].max())

20190729 20210125


In [ ]:
f21['H_DATE'].max()

20210125

In [ ]:
f2020 = f20[f20['H_DATE']>=20201102]
f2021 = f21[f21['H_DATE']>=20201102]

In [ ]:
print(f2020['H_DATE'].value_counts().count(), f2021['H_DATE'].value_counts().count())

13 13


In [ ]:
print(f2020.shape, f2021.shape)

(84448, 106) (54535, 109)


In [ ]:
f_comb = f2020.merge(f2021, on=['ART_CINV', 'H_DATE'], suffixes=('_20','_21'), how='inner')
f = f_comb[['ART_CINV', 'H_DATE', 'FCT_VALUE_20', 'FCT_VALUE_21','REAL_VALUE_TO_FORECAST_21']]

f.tail(10)

,ART_CINV,H_DATE,FCT_VALUE_20,FCT_VALUE_21,REAL_VALUE_TO_FORECAST_21
52354,2849742,20201228,25.867466,22.785,4.0
52355,2849742,20210104,25.947890,20.920,5.0
52356,2849742,20210111,25.997791,24.210,7.0
52357,2849742,20210118,26.073209,26.195,4.0
52358,2849742,20210125,26.158679,25.170,1.0
52359,2861421,20201102,756.603524,510.000,700.0
52360,2861421,20201109,689.917348,510.000,540.0
52361,2861421,20201116,650.350019,510.000,560.0
52362,2861421,20201123,639.206892,510.000,600.0
52363,2861421,20201130,638.123894,510.000,490.0


In [ ]:
f.shape

(52364, 5)

In [ ]:
f = f.merge(all, how='left', on='ART_CINV')
f.head()

,ART_CINV,H_DATE,FCT_VALUE_20,FCT_VALUE_21,REAL_VALUE_TO_FORECAST_21,ART_DESC
0,1919435,20201102,0.0,0.0,0.0,50ML NIV CR REV Q10 HOMME SV1
1,1919435,20201109,0.0,0.0,0.0,50ML NIV CR REV Q10 HOMME SV1
2,1919435,20201116,0.0,0.0,0.0,50ML NIV CR REV Q10 HOMME SV1
3,1919435,20201123,0.0,0.0,0.0,50ML NIV CR REV Q10 HOMME SV1
4,1919435,20201130,0.0,0.0,0.0,50ML NIV CR REV Q10 HOMME SV1


In [ ]:
fk = f2020[f2020['ART_CINV']==2850158]
fk[fk['H_DATE']== 20210118][['ART_CINV', 'H_DATE', 'FCT_VALUE']]

,ART_CINV,H_DATE,FCT_VALUE
26216,2850158,20210118,1845.955566


In [ ]:
fk2 = f2021[f2021['ART_CINV']==2850158]
fk2[fk2['H_DATE']== 20210118][['ART_CINV', 'H_DATE', 'FCT_VALUE','REAL_VALUE_TO_FORECAST']]


,ART_CINV,H_DATE,FCT_VALUE,REAL_VALUE_TO_FORECAST
308705,2850158,20210118,1815.2,1908.0


In [ ]:
fk3 = f[f['ART_CINV']==2850158]
fk3[fk3['H_DATE']== 20210118]

,ART_CINV,H_DATE,FCT_VALUE_20,FCT_VALUE_21,REAL_VALUE_TO_FORECAST_21,ART_DESC
7773,2850158,20210118,1845.955566,1815.2,1908.0,BRANDX GRATED CARROTS 350G SV1


In [ ]:
mse20 = sklearn.metrics.mean_squared_error(f['REAL_VALUE_TO_FORECAST_21'], f['FCT_VALUE_20'])
rmse20 = math.sqrt(mse20)
mae20 = sklearn.metrics.mean_absolute_error(f['REAL_VALUE_TO_FORECAST_21'], f['FCT_VALUE_20'])
mape20 = sklearn.metrics.mean_absolute_percentage_error(f['REAL_VALUE_TO_FORECAST_21'], f['FCT_VALUE_20'])

mse21 = sklearn.metrics.mean_squared_error(f['REAL_VALUE_TO_FORECAST_21'], f['FCT_VALUE_21'])
rmse21 = math.sqrt(mse21)
mae21 = sklearn.metrics.mean_absolute_error(f['REAL_VALUE_TO_FORECAST_21'], f['FCT_VALUE_21'])
mape21 = sklearn.metrics.mean_absolute_percentage_error(f['REAL_VALUE_TO_FORECAST_21'], f['FCT_VALUE_21'])

In [ ]:
print('Forecast from 2020.11.02: MSE =',int(mse20),'  RMSE = ',int(rmse20),'  MAE = ',int(mae20),' MAPE = ', int(mape20))
print('Forecast from 2021.01.27: MSE =',int(mse21),'  RMSE = ',int(rmse21),'  MAE = ',int(mae21),' MAPE = ', int(mape21))

Forecast from 2020.11.02: MSE = 25175362   RMSE =  5017   MAE =  1205  MAPE =  2254876539926654464
Forecast from 2021.01.27: MSE = 21733890   RMSE =  4661   MAE =  1086  MAPE =  2098466716367424512


In [ ]:
f_ar=mape(f[f['ART_CINV']==2850158]['REAL_VALUE_TO_FORECAST_21'], f[f['ART_CINV']==2850158]['FCT_VALUE_21'])
print(f_ar)

0.4295046047344088


In [ ]:
scope=pd.read_csv('scopeItem_list.csv', sep=';')
len(scope)

569

In [ ]:
sc_list = scope['ART_CINV'].to_list()
len(sc_list)

569

In [ ]:
f_scope = f[f['ART_CINV'].isin(sc_list)]

In [ ]:
f_scope.groupby('ART_CINV').count()

,H_DATE,FCT_VALUE_20,FCT_VALUE_21,REAL_VALUE_TO_FORECAST_21,ART_DESC
ART_CINV,,,,,
2196458,13,13,13,13,13
2196581,13,13,13,13,13
2196957,13,13,13,13,13
2196962,13,13,13,13,13
2196969,13,13,13,13,13
...,...,...,...,...,...
2905861,13,13,13,13,13
2905884,13,13,13,13,13
2908125,13,13,13,13,13


In [ ]:
f_sc_list = list(set(f_scope['ART_CINV'].to_list()))
print(len(f_sc_list))

462


In [ ]:
nu =2196581	
fa = f[f['ART_CINV']==nu]
mape_e = mape(fa['REAL_VALUE_TO_FORECAST_21'], fa['FCT_VALUE_21'])
mape_e


0.4794465387608067

In [ ]:
dic_mape21 = {}
list_mape21=[]
for nua in f_sc_list:
  fa = f[f['ART_CINV']==nua]
  mp = mape(fa['REAL_VALUE_TO_FORECAST_21'], fa['FCT_VALUE_21'])
  dic_mape21[nua]=mp
  list_mape21.append(mp)
print(dic_mape21)

dic_mape20 = {}
list_mape20=[]
for nua in f_sc_list:
  fa = f[f['ART_CINV']==nua]
  mp = mape(fa['REAL_VALUE_TO_FORECAST_21'], fa['FCT_VALUE_20'])
  dic_mape20[nua]=mp
  list_mape20.append(mp)
print(dic_mape20)




{2254855: 0.4941089907137511, 2228232: 0.718461674453716, 2572302: 0.39186928004196464, 2224146: 0.6012408694897838, 2598941: 0.401792158866109, 2215985: 0.4781745182025045, 2902067: 0.41501409071311446, 2398259: 0.8455163417018471, 2203701: 0.3849669293352021, 2218038: 0.5291441242197863, 2783287: 0.5775882528710329, 2398266: 1.2884739798935105, 2572352: 0.5574275076112717, 2289728: 0.4234208339874479, 2902085: 0.43686407094202934, 2281550: 0.18890221504953342, 2220113: 5.209798816731314e+18, 2281559: 0.3307325629176898, 2228313: 0.7047147427578059, 2220122: 0.5555829211213014, 2902108: 0.5949108564224727, 2562149: 0.9472602005002966, 2599022: 2.9343607843376764, 2599029: 3.3624890741658655, 2259067: 0.5268475275767384, 2599036: 1.2364567142714438, 2259074: 0.42230104297026394, 2246789: 0.5646315422603008, 2599050: 0.47290514117397703, 2580633: 3.9073229674204943e+18, 2486434: 0.4388382032399382, 2547889: 0.5127156989517005, 2230451: 0.6397599994831844, 2867396: 0.260668779765631, 286

In [ ]:
f_for_scope = pd.DataFrame({'ART_CINV':f_sc_list, 'mape2020':list_mape20, 'mape2021': list_mape21})

In [ ]:
f_for_scope.shape

(462, 3)

In [ ]:
f_for_scopen = f_for_scope.merge(all,how='left', on='ART_CINV')

In [ ]:
f_for_scopen.shape

(462, 4)

In [ ]:
mape_scope = f_for_scopen[['ART_CINV','ART_DESC','mape2020','mape2021']]
mape_scope.head()

,ART_CINV,ART_DESC,mape2020,mape2021
0,2254855,BRANDX BIO BURGER KALE HEMP 200G SV1,0.929954,0.494109
1,2228232,SURIMI STICK+COCKTAIL SAUCE SV1,0.620030,0.718462
2,2572302,BRANDX BIO TRAN.VEGETARIENE NATURE SV1,0.450618,0.391869
3,2224146,FRANKFURTER SAUC 4X70G SV1,0.614972,0.601241
4,2598941,CHICKEN LEG FILLET SV1,0.489454,0.401792


In [ ]:
best_items = mape_scope.sort_values(by='mape2021').head()
worst_items = mape_scope.sort_values(by='mape2021').tail()

In [ ]:
print(best_items)

     ART_CINV                            ART_DESC      mape2020  mape2021
80    2501061               CHICKEN PITA 400G SV1  2.522821e-01  0.140242
380   2280990              PATE PHEASANT 150G SV1  7.368272e+18  0.154984
258   2579450  BRANDX LONDON DOUBLE BEEF 275G SV1  1.307217e-01  0.161276
247   2501614              CHICKEN GYROS 400G SV1  2.591710e-01  0.168141
240   2579422    BRANDX NY CHEESE BURGER 195G SV1  1.763213e-01  0.185873


In [ ]:
print(worst_items)

     ART_CINV                            ART_DESC      mape2020      mape2021
436   2838379           NACHO CHEESE DIP 180G SV1  4.067221e+18  2.964297e+18
29    2580633       BIO SCAMPI WITH TAIL 125G SV1  4.331191e+18  3.907323e+18
265   2247707   TOI 200G CREV GRIS FRAIS BELG SV1  2.821794e+18  3.970096e+18
16    2220113  TOI 100G CREV GRISE BELGES FRA SV1  4.469589e+18  5.209799e+18
412   2834187        MINI KNACK SAUSAGES 100G SV1  6.645554e+18  5.907250e+18


In [ ]:
f[f['ART_CINV'].isin([2501061])]


,ART_CINV,H_DATE,FCT_VALUE_20,FCT_VALUE_21,REAL_VALUE_TO_FORECAST_21,ART_DESC
42973,2501061,20201102,2643.000997,2359.19995,2400.0,CHICKEN PITA 400G SV1
42974,2501061,20201109,2665.126076,2706.81000,2328.0,CHICKEN PITA 400G SV1
42975,2501061,20201116,2689.556981,2404.75000,2334.0,CHICKEN PITA 400G SV1
42976,2501061,20201123,2695.667518,2426.99000,2544.0,CHICKEN PITA 400G SV1
42977,2501061,20201130,2695.667518,2538.35000,2532.0,CHICKEN PITA 400G SV1
42978,2501061,20201207,2695.667518,2394.66294,2226.0,CHICKEN PITA 400G SV1
42979,2501061,20201214,2695.667518,2292.21430,2316.0,CHICKEN PITA 400G SV1
42980,2501061,20201221,2689.001635,2144.00409,1764.0,CHICKEN PITA 400G SV1
42981,2501061,20201228,2689.001635,2514.51117,2094.0,CHICKEN PITA 400G SV1
42982,2501061,20210104,2695.667518,2359.19995,2991.6,CHICKEN PITA 400G SV1


In [ ]:
f[f['ART_CINV'].isin([2834187])]

,ART_CINV,H_DATE,FCT_VALUE_20,FCT_VALUE_21,REAL_VALUE_TO_FORECAST_21,ART_DESC
49824,2834187,20201102,2031.180732,1703.10,1609.8,MINI KNACK SAUSAGES 100G SV1
49825,2834187,20201109,2080.957838,1667.85,1440.0,MINI KNACK SAUSAGES 100G SV1
49826,2834187,20201116,2285.649705,2988.00,1068.0,MINI KNACK SAUSAGES 100G SV1
49827,2834187,20201123,2088.707406,1410.10,384.0,MINI KNACK SAUSAGES 100G SV1
49828,2834187,20201130,2107.069202,1413.85,0.0,MINI KNACK SAUSAGES 100G SV1
49829,2834187,20201207,2122.814182,2988.00,0.0,MINI KNACK SAUSAGES 100G SV1
49830,2834187,20201214,2135.614830,2988.00,0.0,MINI KNACK SAUSAGES 100G SV1
49831,2834187,20201221,2141.367854,1700.15,0.0,MINI KNACK SAUSAGES 100G SV1
49832,2834187,20201228,2148.573178,1561.80,0.0,MINI KNACK SAUSAGES 100G SV1
49833,2834187,20210104,2119.117491,1620.10,0.0,MINI KNACK SAUSAGES 100G SV1


In [ ]:
fsc = f21[f21['ART_CINV'].isin(sc_list)]
(f.shape, fsc.shape)

((52364, 6), (36577, 109))

In [ ]:
fcs = fsc[['ART_CINV', 'H_DATE', 'FCT_VALUE', 'REAL_VALUE_TO_FORECAST']]
fcs.head()


,ART_CINV,H_DATE,FCT_VALUE,REAL_VALUE_TO_FORECAST
726,2196990,20190729,639.90000,637.0
727,2196990,20190805,632.40002,785.0
728,2196990,20190812,662.79999,766.0
729,2196990,20190819,695.55000,694.0
730,2196990,20190826,710.85000,716.0


In [ ]:
fcs.to_csv('f21_tableau.csv')